In [ ]:
# --- Imports ---
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
import joblib
import json

# --- 1. Load saved model and scaler ---
model = load_model("success_predictor_model_optimized.keras", compile=False)

scaler = joblib.load("scaler.pkl")

# Load y_min and y_max for denormalizing the prediction
with open("y_norm_range.json") as f:
    y_min, y_max = json.load(f)

# --- 2. Define new event inputs ---

# 🔢 Numeric features (replace these with your real values)
# --- Define 3 events: bad, average, great ---

new_event_numeric = pd.DataFrame([
    # 💀 BAD — new venue, almost no interaction
    {
        "rating": 2.1,
        "total_events": 5,
        "avg_checkins": 5.0,
        "avg_likes": 10.0,
        "avg_event_rating": 2.5,
        "median_event_rating": 2.4,
        "venue_popularity_tier": 1,
        "checkin_count": 3,
        "like_count": 8,
        "rating_avg": 2.0,
        "rating_count": 3,
        "engagement": 10.5,
        "event_weekday": 2,
        "event_month": 1,
        "start_hour": 13,
        "price_rating_encoded": 0,
    },

    # 💀 BAD — okay artist but poorly attended, low buzz
    {
        "rating": 3.2,
        "total_events": 20,
        "avg_checkins": 12.0,
        "avg_likes": 20.0,
        "avg_event_rating": 3.0,
        "median_event_rating": 2.9,
        "venue_popularity_tier": 2,
        "checkin_count": 10,
        "like_count": 18,
        "rating_avg": 3.0,
        "rating_count": 8,
        "engagement": 28.0,
        "event_weekday": 3,
        "event_month": 2,
        "start_hour": 15,
        "price_rating_encoded": 0,
    },

    # 💀 BAD — cheap, niche, not known, no interaction
    {
        "rating": 1.9,
        "total_events": 8,
        "avg_checkins": 6.0,
        "avg_likes": 12.0,
        "avg_event_rating": 2.3,
        "median_event_rating": 2.2,
        "venue_popularity_tier": 1,
        "checkin_count": 4,
        "like_count": 9,
        "rating_avg": 1.8,
        "rating_count": 4,
        "engagement": 14.0,
        "event_weekday": 1,
        "event_month": 10,
        "start_hour": 16,
        "price_rating_encoded": 0,
    },
])[[
    'rating', 'total_events', 'avg_checkins', 'avg_likes', 'avg_event_rating',
    'median_event_rating', 'venue_popularity_tier', 'checkin_count', 'like_count',
    'rating_avg', 'rating_count','engagement', 'event_weekday', 'event_month', 'start_hour',
    'price_rating_encoded'
]]





# 🏷️ Tag IDs (must be padded to length 6)
tag_lists = [
    [50],        # Unlabeled / unknown
    [19, 22],    # Experimental + Abstract
    [45]         # Spoken poetry / unknown
]

# -- 1. Pad tag input (already correct) --
max_len = 6
tag_input = np.array([
    tags + [0] * (max_len - len(tags)) for tags in tag_lists
])

# -- 2. Scale numeric input once --
scaler = joblib.load("scaler.pkl")
new_numeric_scaled = scaler.transform(new_event_numeric)

# -- 3. Predict with correct input shapes --
pred_scaled = model.predict({
    "numeric_input": new_numeric_scaled,
    "tag_input": tag_input
}).flatten()

# -- 4. Denormalize --
pred_scores = pred_scaled * (y_max - y_min) + y_min

# -- 5. Print results --
for i, score in enumerate(pred_scores):
    print(f" Event → Predicted Success Score: {score:.4f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
 Event → Predicted Success Score: 0.3926
 Event → Predicted Success Score: 0.7083
 Event → Predicted Success Score: 0.7205
